# MNIST Implementaiton with ANN - Improving accuracy #2

In [1]:
# Load necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.utils import to_categorical
from keras.datasets import mnist
import pandas as pd

In [2]:
# Load MNIST handwritten digit data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
#Lets do a min max scaling to make it between 0 & 1. This will be good for us in training.
X_train=(X_train - X_train.min()) /(X_train.max() - X_train.min())
X_test=(X_test - X_test.min()) /(X_test.max() - X_test.min())

# Convert y_train into one-hot format 
y_train_hot = to_categorical(y_train, num_classes=10)
y_test_hot = to_categorical(y_test, num_classes=10)

### Randomly trying with 3 hidden layers with 100 nodes each. Also RELU activation is used.

##### Note : model.fit includes an optional argument batch_size. If unspecified, batch_size will default to 32
##### 60000/32 = 1875

In [3]:
# Create simple Neural Network model
model = Sequential()
model.add(Flatten(input_shape=(28,28)))      # Input layer 784 neurons
model.add(Dense(100, activation='relu'))     # 1st Hidden layer 100 neurons
model.add(Dense(100, activation='relu'))     # 2nd Hidden layer 100 neurons
model.add(Dense(100, activation='relu'))     # 3rd Hidden layer 100 neurons
model.add(Dense(10, activation='softmax'))   # Output layer 10 neurons
model.summary()
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['acc'])
# Train the Neural Network model

model.fit(X_train, y_train_hot, epochs=5,validation_data=(X_test,y_test_hot))
# evaluate the model...Verbose=0 is used to avoid report displays
_, train_acc = model.evaluate(X_train, y_train_hot, verbose=0)
_, test_acc = model.evaluate(X_test, y_test_hot, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
Total params: 99,710
Trainable params: 99,710
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2406 - acc: 0.9279 - val_loss: 0.1315 - v

### Got 97.6% accuracy with in 5 epochs

### Now gonna try with different loss functions and optimizaers.


In [4]:
# Creaing lists of optimizers and loss functions
loss=['categorical_crossentropy','kl_divergence','poisson','sparse_categorical_crossentropy']
opt=['Adadelta','Adagrad','Adam','Adamax','Nadam','RMSprop','SGD']
# Creating a dataframe for keeping results
df=pd.DataFrame(columns =['Loss','Optimizer','Train Accuracy','Test Accuracy'])

In [5]:
# Looping through all the possibilities with 200 Epocs.. This will run for a few hours.
for l in loss:
    for o in opt:
        model=Sequential([  Flatten(input_shape=(28,28)),
                            Dense(100,activation='relu'),
                            Dense(100,activation='relu'),
                            Dense(100,activation='relu'),
                            Dense(10,activation='softmax')  ])                         
        model.compile(optimizer=o, loss=l, metrics=['acc'])
        if l == 'sparse_categorical_crossentropy': # No one hot encoding required for this
            model.fit(X_train,y_train,epochs=200,validation_data=(X_test,y_test),verbose=0)
            _, train_acc = model.evaluate(X_train, y_train, verbose=0)
            _, test_acc = model.evaluate(X_test, y_test, verbose=0)
        else :
            model.fit(X_train,y_train_hot,epochs=200,validation_data=(X_test,y_test_hot),verbose=0)
            _, train_acc = model.evaluate(X_train, y_train_hot, verbose=0)
            _, test_acc = model.evaluate(X_test, y_test_hot, verbose=0)
        print('Loss: %s, Optimizer: %s, Train: %.3f, Test: %.3f' % (l,o,train_acc*100, test_acc*100))
        df = df.append({'Loss':l,'Optimizer':o,'Train Accuracy':train_acc*100,'Test Accuracy':test_acc*100}, ignore_index=True)  

Loss: categorical_crossentropy, Optimizer: Adadelta, Train: 93.742, Test: 93.680
Loss: categorical_crossentropy, Optimizer: Adagrad, Train: 97.620, Test: 96.780
Loss: categorical_crossentropy, Optimizer: Adam, Train: 99.908, Test: 97.780
Loss: categorical_crossentropy, Optimizer: Adamax, Train: 100.000, Test: 97.840
Loss: categorical_crossentropy, Optimizer: Nadam, Train: 99.923, Test: 98.060
Loss: categorical_crossentropy, Optimizer: RMSprop, Train: 99.963, Test: 97.960
Loss: categorical_crossentropy, Optimizer: SGD, Train: 100.000, Test: 97.720
Loss: kl_divergence, Optimizer: Adadelta, Train: 93.622, Test: 93.800
Loss: kl_divergence, Optimizer: Adagrad, Train: 97.345, Test: 96.490
Loss: kl_divergence, Optimizer: Adam, Train: 99.955, Test: 98.100
Loss: kl_divergence, Optimizer: Adamax, Train: 99.998, Test: 97.950
Loss: kl_divergence, Optimizer: Nadam, Train: 99.872, Test: 97.880
Loss: kl_divergence, Optimizer: RMSprop, Train: 99.888, Test: 97.790
Loss: kl_divergence, Optimizer: SGD, T

In [6]:
# Display sorted df
df.sort_values(['Test Accuracy'],ascending=False,ignore_index=True)

,Loss,Optimizer,Train Accuracy,Test Accuracy
0,sparse_categorical_crossentropy,Adam,99.968332,98.240000
1,kl_divergence,Adam,99.954998,98.100001
2,poisson,Adam,99.951667,98.070002
3,categorical_crossentropy,Nadam,99.923331,98.060000
4,poisson,Nadam,99.940002,97.970003
5,categorical_crossentropy,RMSprop,99.963331,97.960001
6,kl_divergence,Adamax,99.998331,97.950000
7,sparse_categorical_crossentropy,Nadam,99.860001,97.920001
8,poisson,Adamax,99.998331,97.899997
9,poisson,RMSprop,99.966669,97.890002


# We were able to get 98.24% accuracy